In [1]:
library(tidyverse)
library(ggupset)

rslt <- '../results/snp_models/'

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
read_models <- function(x, method) {
    read_delim(paste0(rslt, x), delim = ' ', col_types = 'cc') %>% 
        mutate(uniq_snp_id = cbind(SNP1, SNP2) %>% apply(1, sort) %>% apply(2, paste, collapse = "_"),
               method = method) %>%
        select(method, uniq_snp_id)
}

scientific_10 <- function(x) {
  parse(text=gsub("e", " %*% 10^", scales::scientific_format()(x)))
}

models_overlap <- bind_rows(read_models('physical.txt', 'Physical'),
                            read_models('eqtl.txt', 'eQTL'),
                            read_models('chromatin.txt', 'Chromatin'),
                            read_models('eqlt_chrom.txt', 'eQTL + Chromatin'),
                            read_models('eqtl_chrom_phys.txt', 'eQTL + Chromatin + Physical')) %>%
    group_by(uniq_snp_id) %>%
    summarize(methods = list(method)) %>%
    ggplot(aes(x = methods)) +
        geom_bar() +
        labs(x = 'Protocols', y = '# SNP models') +
        scale_x_upset() +
        scale_y_log10(label=scientific_10)

In [3]:
ggsave(models_overlap, filename = 'fig/models_comparison.pdf', width=10, height=8, bg = "transparent")